In [ ]:
!pip install bitsandbytes transformers accelerate

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

base_model_name = "unsloth/llama-2-7b-bnb-4bit"
#Select from follow
#unsloth/mistral-7b-bnb-4bit
#unsloth/llama-2-7b-bnb-4bit
#unsloth/qwen2-7b-bnb-4bit
#unsloth/qwen3-14b-bnb-4bit
max_output_lines = 10

# Load model
model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map='auto',
    load_in_8bit=True,
)
model.bfloat16()

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name)




In [ ]:
# Initialize a counter for outputs
output_counter = 1

# Loop to continuously take user input and log input-output pairs
while True:
    input_text = input("Enter your prompt (or type 'exit' to quit): ")
    if input_text.lower() == "exit":
        break

    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(model.device)
    outputs = model.generate(input_ids, max_new_tokens=1000)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    lines = generated_text.split('\n')[:max_output_lines]
    limited_text = '\n'.join(lines)

    print(f"Generated Output {output_counter}:")
    print(limited_text)
    print("=" * 50)  # Separator for readability

    # Log input and output to a file
    with open("conversation_log.txt", "a", encoding="utf-8") as log_file:
        log_file.write(f"Output {output_counter}:\n")
        log_file.write("User Input:\n")
        log_file.write(input_text + "\n\n")
        log_file.write("Model Output:\n")
        log_file.write(limited_text + "\n")
        log_file.write("=" * 50 + "\n\n")

    output_counter += 1  # Increment counter
